## Introducción a la Visión Computacional
## Tarea 4
-----------------------
<div align="right">
Fecha de Entrega: Martes 17, Mayo 2022.
</div>

1. Proponer mejoras al sistema diseñado e implementado originalmete.

2. Volver a investigar y averiguar si hay algún algoritmo entrenado o bien si se puede extender el algoritmo utilizado trabajado en la Tarea 2.

3. Escribir un informe o entregar el notebook con comentarios y documentación, incluyendo el análisis (comentando) de la comparación con los resultados de la Tarea 2. Notar que pudiera ser que el resultado es inferior al de la Tarea 2, lo importante es el comentario o discusión del por qué se mejoró o empeoraron los nuevos resultados.

# Base de datos de patentes de vehiculos de Chile

Base de datos recopilada por mi, en imagenes de Google, las cuales fueran fotografias de patentes de vehiculos chilenos, con esto me gustaria realizar un posterior analisis, obteniendo las letras y numeros para consultar en [autoseguro.gob.cl](https://www.autoseguro.gob.cl) de carabineros y policia de investigaciones, con el objetivo de verificar si un vehiculo tiene encargo por robo.

En este caso para el pre-procesamiento de imagenes, se aplica el filtro sobel, para poder obtener la deteccion de bordes, con esto podemos tener la capacidad de detectar medianamente bien las letras y numeros de las patentes. Obteniendo estos datos, podria verificar si estos estan con encargo por robo y asi poder informar que el vehiculo fue visto por ultima vez en una locacion especifica

## Definicion funciones para pipeline de procesamiento, conversion y reconocimiento de imagenes
Define funciones intermedias para ser referenciadas desde funcion de ejecucion de pipeline
- obtencion de imagenes desde classpath de proyecto ```get_image_list_from_path(path, extension)```
- pre-procesamiento de imagenes obtenidas en punto anterior con ```save_resized_images_resx_resy```
- procesamiento de imagenes con filtro sobel ```sobel_filter```
- reconocimiento de figuras en imagenes ```detect_vehicle_license_plate```
- reconocimiento optico de caracteres, para llevar a archivo de texto las patentes reconocidas ```ocr_vehicle_license_plate```

In [136]:
import os

import numpy as np
from PIL import Image
from numpy import *
from scipy.ndimage import sobel
from matplotlib import pyplot as plt
import cv2
import time
from pathlib import Path
import shutil
import pytesseract
import glob

# Functiones utilitarias

In [137]:
def stopwatch(sec, function_name):
  mins = sec // 60
  sec = sec % 60
  hours = mins // 60
  mins = mins % 60
  print("Time Lapsed = {0}:{1}:{2} in function '{3}'".format(int(hours), int(mins), sec, function_name))

In [138]:
def delete_dir(path_directory):
    dir_path = Path(path_directory)
    if dir_path.exists() and dir_path.is_dir():
        shutil.rmtree(dir_path)

In [139]:
def delete_file(path_file):
    os.remove(path_file)

# Funciones de pipeline

In [140]:
'''
Obtiene las imagenes definidas en el classpath del proyecto, con los formatos que se establecen en el segundo parametro
    - path: direccion donde estaran alojadas las imagenes a procesar
    - extension: lista de extensiones que seran tomadas en cuenta para obtencion de imagenes
    - limit: limite de imagenes que obtendra desde el parametro path
'''
def get_image_list_from_path(path, extension, limit = -1):
    l = []
    for ex in extension:
        for f in os.listdir(path):
            if f.endswith(ex):
                l.append(os.path.join(path,f))

    if limit != -1:
        return l[:limit]

    return l

In [141]:
#img_from_path = get_image_list_from_path("patentes", [".png", ".jpeg"])

In [142]:
'''
Guarda las imagenes que ya fueron guardadas en el classpath, con una resolucion diferente
    - x: resolucion al eje x
    - y: resolucion al eje y
    - img_list: lista de imagenes que seran procesadas a la resolucion pasada en los parametros x e y
'''
def save_resized_images_resx_resy(x, y, img_list):
    os.makedirs("out/out{}x{}/patentes".format(x, y), exist_ok=True)

    for img in img_list:
        Image.open(img).convert('RGB').resize((x, y))\
            .save("out/out{}x{}/{}".format(x, y, img))

    return get_image_list_from_path("out/out{}x{}/patentes".format(x, y), [".png", ".jpeg"])

In [143]:
#resized_images_512_512 = save_resized_images_resx_resy(512, 512, img_from_path)

In [144]:
'''
De las imagenes que sean pasadas por el parametro 'img_list' (lista de paths), seran convertidas y guardadas con el filtro sobel (reconocimiento de bordes)
    - img_list: lista de imagenes que seran convertidas y guardadas con el filtro sobel
'''
def sobel_filter(img_list):

    os.makedirs("out/out_sobel/patentes", exist_ok=True)

    for img in img_list:
        im = array(Image.open(img).convert('L'), 'f')
        imx = zeros(im.shape)
        sobel(im, 1, imx)

        imy = zeros(im.shape)
        sobel(im, 0, imy)

        magnitude = sqrt(imx**2+imy**2)
        plt.imsave("out/out_sobel/{}".format("/".join(img.split("/")[-2:])), arr = magnitude, cmap = plt.get_cmap('gray'))

    return get_image_list_from_path("out/out_sobel/patentes", [".png", ".jpeg"])

In [145]:
#sobel_filter_images = sobel_filter(resized_images_512_512)

In [146]:
'''
Funcion orientada a reconocimiento de figuras en imagenes pasadas por el parametro 'img_list' (lista de paths)
Las imagenes quedaran con bordes en rojo
    - img_list: lista de imagenes que seran convertidas y guardadas con reconocimiento de figuras
'''
def detect_vehicle_license_plate(img_list):

    os.makedirs("out/out_detection/patentes", exist_ok=True)
    for img_path in img_list:

        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, threshold = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(
            threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        i = 0
        for contour in contours:
            if i == 0:
                i = 1
                continue
            approx = cv2.approxPolyDP(
                contour, 0.01 * cv2.arcLength(contour, True), True)
            cv2.drawContours(img, [contour], 0, (0, 0, 255), 5)
            M = cv2.moments(contour)
            if M['m00'] != 0.0:
                x = int(M['m10']/M['m00'])
                y = int(M['m01']/M['m00'])

        cv2.imwrite('out/out_detection/{}'.format("/".join(img_path.split("/")[-2:])), img)

In [147]:
#detect_vehicle_license_plate(sobel_filter_images)

In [153]:
def ocr_vehicle_license_plate(img_list):
    list_license_plates = []
    predicted_license_plates = []

    for img_l in img_list:
        license_plate_file = img_l
        license_plate, _ = os.path.splitext(license_plate_file)

        list_license_plates.append(license_plate)

        image = cv2.imread(img_l)
        predicted_result = pytesseract.image_to_string(image, lang = 'eng', config = '--oem 3 --psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')

        filter_predicted_result = "".join(predicted_result.split()).replace(":", "").replace("-", "")
        predicted_license_plates.append(filter_predicted_result)


    textfile = open("out/vehicle_license_plates.txt", "w")
    for element in predicted_license_plates:
        if element != '':
            textfile.write(element + "\n")
    textfile.close()

    #return predicted_license_plates

In [154]:
#ocr_vehicle_license_plate(sobel_filter_images)

In [155]:
'''
Funcion de ejecucion de proceso de recoleccion, pre-procesamiento, procesamiento, deteccion de figuras y reconocimiento optico de caracteres (OCR) para patentes de vehiculos

    - limit: define cual es el numero de imagenes con que trabajara en todo el flujo. Por defecto es -1
    - res_x: resolucion al eje x, por defecto 512
    - res_y: resolucion al eje y, por defecto 512

NOTA: Esta funcion generara un reporte de demora en cada uno de las funciones de ejecucion,
como tambien de la demora general en el proceso

'''
def pipeline(limit = -1, res_x = 512, res_y = 512):
    start = time.time()

    elapsed_f_preconditions = None
    elapsed_f_get_image_list_from_path = None
    elapsed_f_save_resized_images_resx_resy = None
    elapsed_f_sobel_filter = None
    elapsed_f_detect_vehicle_license_plate = None
    elapsed_f_ocr_vehicle_license_plate = None

    print("-----------------------")
    print("Pre-conditions")
    try:
        start_preconditions = time.time()
        delete_dir('out')
        delete_file('out/vehicle_license_plates.txt')
        end_preconditions = time.time()
        elapsed_f_preconditions = end_preconditions - start_preconditions
        stopwatch(elapsed_f_preconditions, 'pre-conditions block')
    except:
      print("An exception was throw executing pre-conditions block")
    print("-----------------------")

    print("Obtaining images")
    img_from_path = None
    try:
        start_f_get_image_list_from_path = time.time()
        img_from_path = get_image_list_from_path("patentes", [".png", ".jpeg"], limit)
        end_f_get_image_list_from_path = time.time()
        elapsed_f_get_image_list_from_path = end_f_get_image_list_from_path - start_f_get_image_list_from_path
        stopwatch(elapsed_f_get_image_list_from_path, 'get_image_list_from_path')
    except:
      print("An exception was throw executing 'get_image_list_from_path' function")
    print("-----------------------")

    print("Resolution adjust to {0}x{1}".format(res_x, res_y))
    resized_images = None
    try:
        start_f_save_resized_images_resx_resy = time.time()
        resized_images = save_resized_images_resx_resy(res_x, res_y, img_from_path)
        end_f_save_resized_images_resx_resy = time.time()
        elapsed_f_save_resized_images_resx_resy = end_f_save_resized_images_resx_resy - start_f_save_resized_images_resx_resy
        stopwatch(elapsed_f_save_resized_images_resx_resy, 'save_resized_images_resx_resy')
    except:
      print("An exception was throw executing 'save_resized_images_resx_resy' function")
    print("-----------------------")

    print("Sobel filter")
    sobel_filter_images = None
    try:
        start_f_sobel_filter = time.time()
        sobel_filter_images = sobel_filter(resized_images)
        end_f_sobel_filter = time.time()
        elapsed_f_sobel_filter = end_f_sobel_filter - start_f_sobel_filter
        stopwatch(elapsed_f_sobel_filter, 'sobel_filter')
    except:
      print("An exception was throw executing 'sobel_filter' function")
    print("-----------------------")

    print("Shape Detection")
    try:
        start_f_detect_vehicle_license_plate = time.time()
        detect_vehicle_license_plate(sobel_filter_images)
        end_f_detect_vehicle_license_plate = time.time()
        elapsed_f_detect_vehicle_license_plate = end_f_detect_vehicle_license_plate - start_f_detect_vehicle_license_plate
        stopwatch(elapsed_f_detect_vehicle_license_plate, 'detect_vehicle_license_plate')
    except:
      print("An exception was throw executing 'detect_vehicle_license_plate' function")
    print("-----------------------")

    print("OCR Vehicle license plate")
    try:
        start_f_ocr_vehicle_license_plate = time.time()
        ocr_vehicle_license_plate(img_from_path)
        end_f_ocr_vehicle_license_plate = time.time()
        elapsed_f_ocr_vehicle_license_plate = end_f_ocr_vehicle_license_plate - start_f_ocr_vehicle_license_plate
        stopwatch(elapsed_f_ocr_vehicle_license_plate, 'ocr_vehicle_license_plate')
    except:
      print("An exception was throw executing 'ocr_vehicle_license_plate' function")
    print("-----------------------")
    print("")

    elapsed = [
        elapsed_f_preconditions,
        elapsed_f_get_image_list_from_path,
        elapsed_f_save_resized_images_resx_resy,
        elapsed_f_sobel_filter,
        elapsed_f_detect_vehicle_license_plate,
        elapsed_f_ocr_vehicle_license_plate
    ]

    print("Metrics execution")
    print("Max time execution: {0}".format(np.amax(elapsed)))
    print("Min time execution: {0}".format(np.amin(elapsed)))
    print("Avg time execution: {0}".format(np.average(elapsed)))
    print("Std-Dev(+-) execution: {0}".format(np.std(elapsed)))
    print("")


    end = time.time()
    stopwatch(end - start, 'pipeline')

In [156]:
pipeline()

-----------------------
Pre-conditions
An exception was throw executing pre-conditions block
-----------------------
Obtaining images
Time Lapsed = 0:0:0.00092315673828125 in function 'get_image_list_from_path'
-----------------------
Resolution adjust to 512x512
Time Lapsed = 0:0:1.2604649066925049 in function 'save_resized_images_resx_resy'
-----------------------
Sobel filter
Time Lapsed = 0:0:1.8754820823669434 in function 'sobel_filter'
-----------------------
Shape Detection
Time Lapsed = 0:0:0.6406159400939941 in function 'detect_vehicle_license_plate'
-----------------------
OCR Vehicle license plate
Time Lapsed = 0:0:13.70766282081604 in function 'ocr_vehicle_license_plate'
-----------------------

Metrics execution


TypeError: '>=' not supported between instances of 'NoneType' and 'float'

In [ ]:
#pipeline(res_x=512, res_y=192)